In [53]:
def getAttributes(expression):
    expression = expression.split("(")[1:] 
    expression = "(".join(expression) 
    expression = expression.split(")")[:-1]
    expression = ")".join(expression)
    attributes = expression.split(',')
    return attributes

def getPredicate(expression):
    return expression.split("(")[0]

def isConstant(char):
    return char.isupper() and len(char) == 1

def isVariable(char):
    return char.islower() and len(char) == 1

def substitute(exp, old, new):
    attributes = getAttributes(exp)
    predicate = getPredicate(exp)
    for index, val in enumerate(attributes):
        if val == old:
            attributes[index] = new
    return predicate + "(" + ",".join(attributes) + ")"

def apply(exp, substitutions):
    for substitution in substitutions:
        new, old = substitution  
        exp = substitute(exp, old, new)
    return exp

def checkOccurs(var, exp):
    if exp.find(var) == -1:
        return False
    return True

def getFirstAttribute(expression):
    attributes = getAttributes(expression)
    return attributes[0]

def getRemaining(expression):
    predicate = getPredicate(expression)
    attributes = getAttributes(expression)
    newExpression = predicate + "(" + ",".join(attributes[1:]) + ")"
    return newExpression

def replaceNestedSubs(substitutions):
    for i, subs in enumerate(substitutions):
        new, old = subs
        for j, subs_val in enumerate(substitutions):
            new_val, old_val = subs_val
            if i == j:
                continue
            if old in new_val:
                substitutions[j] = (substitute(new_val, old, new), old_val)
    return substitutions

In [54]:
def unify(exp1, exp2):
    if exp1 == exp2:
        return []
    elif isConstant(exp1) and isConstant(exp2):
        if exp1 != exp2:
            print(f"{exp1} and {exp2} are constants. Cannot be unified")
            return []
    elif isConstant(exp1):
        return [(exp1, exp2)]
    elif isConstant(exp2):
        return [(exp2, exp1)]
    elif isVariable(exp1):
        return [(exp2, exp1)] if not checkOccurs(exp1, exp2) else []
    elif isVariable(exp2):
        return [(exp1, exp2)] if not checkOccurs(exp2, exp1) else []
    elif getPredicate(exp1) != getPredicate(exp2):
        print(f"{getPredicate(exp1)} and {getPredicate(exp2)} do not match. Cannot be unified")
        return []
    elif len(getAttributes(exp1)) != len(getAttributes(exp2)):
        print(f"Length of attributes {len(getAttributes(exp1))} and {len(getAttributes(exp2))} do not match. Cannot be unified")
        return []

    # Unify first attributes
    firstAttr1 = getFirstAttribute(exp1)
    firstAttr2 = getFirstAttribute(exp2)
    initialSubstitution = unify(firstAttr1, firstAttr2)
    if not initialSubstitution:
        return []
    if len(getAttributes(exp1)) == 1:
        return initialSubstitution

    remainingAttr1 = getRemaining(exp1)
    remainingAttr2 = getRemaining(exp2)
    if initialSubstitution != []:
        remainingAttr1 = apply(remainingAttr1, initialSubstitution)
        remainingAttr2 = apply(remainingAttr2, initialSubstitution)
    remainingSubstitution = unify(remainingAttr1, remainingAttr2)
    if not remainingSubstitution:
        return []

    substitutions = replaceNestedSubs(initialSubstitution + remainingSubstitution)
    return substitutions

In [56]:
exp1 = input("Expresssion 1: ")
# exp1 = 'Knows(John,f(x))'
exp2 = input("Expresson 2: ")
# exp2 = 'Knows(y,f(g(y)))'

substitutions = unify(exp1, exp2)

print("Substitutions:")
print([' / '.join(substitution) for substitution in substitutions])

Expresssion 1: Knows(John,f(x))
Expresson 2: Knows(y,f(g(y)))
Substitutions:
['John / y', 'g(John) / x']


In [57]:
exp1 = input("Expresssion 1: ")
# exp1 = 'Student(x)'
exp2 = input("Expresssion 2: ")
# exp2 = 'Teacher(y)'

substitutions = unify(exp1, exp2)

print("Substitutions:")
print([' / '.join(substitution) for substitution in substitutions])

Expresssion 1: Student(x)
Expresssion 2: Teacher(y)
Student and Teacher do not match. Cannot be unified
Substitutions:
[]


In [58]:
# Case 1) Predicates are different 

exp1 = input("Expresssion 1: ")
exp2 = input("Expresssion 2: ")
# exp1 = 'Knows(x, John)'
# exp2 =  'Loves(Bob, y)'

substitutions = unify(exp1, exp2)

print("Substitutions:")
print([' / '.join(substitution) for substitution in substitutions])

Expresssion 1: Knows(x, John)
Expresssion 2: Loves(Bob, y)
Knows and Loves do not match. Cannot be unified
Substitutions:
[]


In [59]:
# Case 2) Mismatch in Number of arguments 

exp1 = input("Expresssion 1: ")
exp2 = input("Expresssion 2: ")
# exp1 = 'Knows(x, John, Bob)'
# exp2 =  'Knows(Alice, y)'

substitutions = unify(exp1, exp2)

print("Substitutions:")
print([' / '.join(substitution) for substitution in substitutions])

Expresssion 1: Knows(x, John, Bob)
Expresssion 2: Knows(Alice, y)
Length of attributes 3 and 2 do not match. Cannot be unified
Substitutions:
[]


In [60]:
# Case 3) If the arguments are constants

exp1 = input("Expresssion 1: ")
exp2 = input("Expresssion 2: ")
# exp1 = 'Knows(x, John)'
# exp2 =  'Knows(Bob, Jack)'

substitutions = unify(exp1, exp2)

print("Substitutions:")
print([' / '.join(substitution) for substitution in substitutions])

Expresssion 1: Knows(x, John)
Expresssion 2: Knows(Bob, Jack)
 John and  Jack do not match. Cannot be unified
Substitutions:
[]
